In [5]:
!pip install langchain  openai langchain_core tiktoken langchain_community -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.5 MB/s eta 0:00:00


In [3]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [16]:
AgentType.ZERO_SHOT_REACT_DESCRIPTION??

In [7]:
# 示例 1: 简单的数学计算

from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.llms import OpenAI

## 定义核心的简单计算函数
def calculate(expression):
    return eval(expression) # eval("15 * 3 + 7") = 52

## 这里只有一个工具
tools = [
    Tool(
        name="Calculator",
        func=calculate,
        description="Useful for performing mathematical calculations"
    )
]

llm = OpenAI(temperature=0)

## 初始化 agent
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

## 运行 agent
result = agent.run("What is the result of 15 * 3 + 7?")
print(result)



> Entering new AgentExecutor chain...
 I should use the order of operations to solve this problem.
Action: Calculator
Action Input: 15 * 3 + 7
Observation: 52
Thought: I now know the final answer.
Final Answer: 52

> Finished chain.
52


In [10]:
eval("15 * 3 + 7")

52

In [12]:
# 示例 2: 天气查询和决策
def get_weather(location):
    # 这里应该是实际的天气 API 调用,为了示例我们使用模拟数据
    return f"The weather in {location} is sunny with a high of 25°C."

tools = [
    Tool(
        name="WeatherAPI",
        func=get_weather,
        description="Useful for getting weather information for a specific location"
    ),
    Tool(
        name="Calculator",
        func=calculate,
        description="Useful for performing mathematical calculations")
]

agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

result = agent.run("What's the weather like in London? Should I bring an umbrella?")
print(result)



> Entering new AgentExecutor chain...
 I should use the WeatherAPI to get the weather information for London
Action: WeatherAPI
Action Input: London
Observation: The weather in London is sunny with a high of 25°C.
Thought: Based on the weather information, I don't need to bring an umbrella.
Final Answer: The weather in London is sunny with a high of 25°C, no need to bring an umbrella.

> Finished chain.
The weather in London is sunny with a high of 25°C, no need to bring an umbrella.


In [15]:
# 示例 3: 多步骤任务
def search_database(query):
    # 模拟数据库搜索
    if "population" in query.lower():
        return "The population of New York City is approximately 8.4 million."
    return "No relevant information found."

# def calculate_percentage(numbers):
#     # 简单的百分比计算
#     return eval(numbers) * 100

def calculate_percentage(numbers):
    # 移除 "million" 并将字符串转换为浮点数，然后计算百分比
    num1, num2 = numbers.split(',')
    num1 = float(num1.replace('million', '').strip()) * 1_000_000
    num2 = float(num2.replace('million', '').strip()) * 1_000_000
    return (num1 / num2) * 100

tools = [
    Tool(
        name="DatabaseSearch",
        func=search_database,
        description="Useful for finding factual information"
    ),
    Tool(
        name="PercentageCalculator",
        func=calculate_percentage,
        description="Calculates percentages"
    )
]

agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

result = agent.run("What percentage of the US population lives in New York City? The US population is about 330 million.")
print(result)



> Entering new AgentExecutor chain...
 I should use the PercentageCalculator tool to calculate the percentage.
Action: PercentageCalculator
Action Input: 8.4 million, 330 million
Observation: 2.5454545454545454
Thought: I now know the percentage of the US population that lives in New York City.
Final Answer: Approximately 2.55% of the US population lives in New York City.

> Finished chain.
Approximately 2.55% of the US population lives in New York City.
